<a href="https://colab.research.google.com/github/babelfishz/colab-models/blob/master/grabber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#import os,sys
#from PIL import Image

#import urllib.request

from pathlib import Path
import shutil
import requests
import time

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_floras():
  url ="http://www.ocalendar.com.cn/api/photo/test/"
  userId = 'eyJpdiI6InNIR2ZEZUNtMjR3V292bjlBNWJoeFE9PSIsInZhbHVlIjoiUlwvUk1wcnFkUEpYNktuVXY3dnRadzNNTk0yNzhmOVcrWDNXQUtFVXY5dE1BR0MrSWVuQjRlUlQ1aXBja3VtYWwiLCJtYWMiOiJmZjYzNWY1ZWY3OTU5NTljNWQ2ZmIxNDcxOGZkYzI0N2U0ZDUyZWYwNGQwYzQ3YTlhNzVmOWUyM2I5M2MwN2Y5In0'
  
  PARAMS = {
    'userId': userId,
  } 
  response = requests.get(url=url, params = PARAMS)
  return response.json()

def read_from_file():
  index = 0
  try:
    with open('/content/gdrive/My Drive/progress/grabber.txt', "r") as f:
      for line in f:
        key, value = line.strip().split()
        index = int(value)
  except:
    index = 0
  return index

def write_to_file(index):
    with open('/content/gdrive/My Drive/progress/grabber.txt', "w") as f:
      string = 'index' + " " + str(index)
      f.write(string + "\n")

def grab_files(name):
  
  url ="http://www.ocalendar.com.cn/api/photo/triple/name/" + name
  userId = 'eyJpdiI6InNIR2ZEZUNtMjR3V292bjlBNWJoeFE9PSIsInZhbHVlIjoiUlwvUk1wcnFkUEpYNktuVXY3dnRadzNNTk0yNzhmOVcrWDNXQUtFVXY5dE1BR0MrSWVuQjRlUlQ1aXBja3VtYWwiLCJtYWMiOiJmZjYzNWY1ZWY3OTU5NTljNWQ2ZmIxNDcxOGZkYzI0N2U0ZDUyZWYwNGQwYzQ3YTlhNzVmOWUyM2I5M2MwN2Y5In0'
  

  PARAMS = {'userId': userId}
  res = requests.get(url=url, params = PARAMS)
  resdata = res.json()
  floras = resdata['floras']['data']
  print(len(floras)," files to grab")

  files =[]
  
  
  for floraData in floras:
    filePath = floraData['filePath']
    fileName = floraData['fileName']
    #tempfile = "temp" + fileName
    
    imageUrl =  'http://www.ocalendar.com.cn/'+ filePath + fileName
    print(imageUrl)
    try:
      content = requests.get(imageUrl).content
      with open(fileName, 'wb') as f:
        f.write(content)
        files.append(fileName)
    except:
      print("something wrong")
      continue

  return files

def copy_files(files, dst):
  Path(dst).mkdir(parents=True, exist_ok=True)
  for file in files:
    try:
      shutil.copyfile(file, dst+file)
      #print(file, dst+file) 
    except:
      print(dst + file+ "copy failed")
      continue

# plots images with labels within jupyter notebook
def plots(files, figsize=(24,12), cols=10, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    #cols = len(files)//rows if len(files) % 2 == 0 else len(files)//rows + 1
    rows = len(files)//cols + 1
    for i in range(len(files)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        img=mpimg.imread(files[i])
        plt.imshow(img)
        plt.show


In [0]:
def main():

  floras = get_floras()
  
  start = read_from_file() 
  
  for index, flora in enumerate(floras[start:]):
    name = flora['floraName']
    print(index+start+1, name)
    
    files = grab_files(name)
    
    dst ='/content/gdrive/My Drive/test/' + name + '/'
    Path(dst).mkdir(parents=True, exist_ok=True)
    copy_files(files, dst)

    write_to_file(start + index + 1)

    #plots(files)

if __name__ == '__main__':
    main()

In [0]:

files = grab_files("齿瓣石豆兰")


In [0]:
plots(files)